In [2]:
%pip install lingua-language-detector

     ---------------------------------------- 73.3/73.3 MB 9.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm
import sqlite3
import re
from lingua import Language, LanguageDetectorBuilder

vocab = np.loadtxt('model_vocab.txt', dtype=str).tolist()

db_path = "Reddit Data/database.sqlite"

batch_size = 10000
skip_gram = 1  # 0 for CBOW, 1 for skip-gram

detector = LanguageDetectorBuilder.from_all_languages().build()

print('init model')
model = Word2Vec([vocab], vector_size=100, window=5, sg=skip_gram, epochs=1)

def preprocess_text(text):
    return re.sub(r'[^\w\s]', '', str(text).lower()).split()

def is_english(text):
    detected_language = detector.detect_language_of(text)
    return detected_language == Language.ENGLISH

def train_model(chunk):
    sentences = [preprocess_text(text) for text in chunk['body'] if is_english(text)]
    model.train(sentences, total_examples=len(sentences), epochs=5)

def fetch_and_train(low):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(f"SELECT body FROM May2015 LIMIT {batch_size} OFFSET {low}", conn)
    conn.close()
    train_model(df)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM May2015")
# total_rows = cursor.fetchone()[0]
total_rows = 100000
conn.close()

# Create a list of tasks for each batch
tasks = [(low) for low in range(0, total_rows, batch_size)]

# Use ProcessPoolExecutor to parallelize the tasks
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(fetch_and_train, tasks), total=len(tasks)))

# Save the model
if skip_gram == 0:
    model.save("reddit_cbow.model")
else:
    model.save("reddit_sg.model")

init model


100%|██████████| 10/10 [01:53<00:00, 11.36s/it]


In [3]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
df = pd.read_sql_query(f"SELECT body FROM May2015 LIMIT {10000} OFFSET {100000}", conn)
conn.close()
average_length = df['body'].str.len().mean()
print(f"Average length of comments: {average_length:.2f}")


Average length of comments: 137.76
